## Imports

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import date, timedelta, datetime

import os

## Load dataframe from newest file

In [ ]:
if not os.path.exists('newest'):
    os.makedirs('newest')
    
for file in os.listdir('newest'):
    if "csv" in file:
        print("Loading "+file+" ...")

df_mx = pd.read_csv('./newest/'+file, encoding = "ISO-8859-1")
print(file+" Loaded")

## Positivity by Week

In [ ]:
df = df_mx

# df = df.loc[df['ENTIDAD_RES'] == 2] #entidad = Baja California
# df = df.loc[df['MUNICIPIO_RES'] == 2] # municipio = mexicali

df = df.loc[(df['RESULTADO_LAB'] == 1) | (df['RESULTADO_LAB'] == 2)| (df['RESULTADO_LAB'] == 3)]

df = df[['FECHA_SINTOMAS','RESULTADO_LAB']]

df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'])
df['SEMANA'] = df['FECHA_SINTOMAS'].dt.week
df['AÑO'] = df['FECHA_SINTOMAS'].dt.year

df.loc[df['RESULTADO_LAB'] == 1, 'CONFIRMADO'] = ['1']
df.loc[df['RESULTADO_LAB'] == 2, 'NEGATIVO'] = ['1']
df.loc[df['RESULTADO_LAB'] == 3, 'SOSPECHOSO'] = ['1']

df = df.groupby(["SEMANA",'AÑO'],as_index=False).count()

df['POSITIVIDAD'] = (df['CONFIRMADO']/df['RESULTADO_LAB'])
df['POSITIVIDAD_MAXIMA'] = ((df['CONFIRMADO']+df['SOSPECHOSO'])/df['RESULTADO_LAB'])
df['POSITIVIDAD_MINIMA'] = (df['CONFIRMADO']/(df['RESULTADO_LAB']+df['SOSPECHOSO']))

df_p = df.loc[df['SOSPECHOSO'] == 0] #remove weeks with cases with pending result
df_p

## Active Cases of Mexico

In [ ]:
df = df_mx
last_positivity = df_p[-1:]['POSITIVIDAD'].iloc[0]

df = df.loc[(df['RESULTADO_LAB'] == 1) | (df['RESULTADO_LAB'] == 3)] # resultado = positivo SARS-COV-2 o Pendiente

# df = df = df.loc[df['ENTIDAD_RES'] == 2] #entidad = Baja California

# df = df.loc[df['MUNICIPIO_RES'] == 2] # municipio = mexicali

df = df[['FECHA_SINTOMAS','RESULTADO_LAB']]

df.loc[df['RESULTADO_LAB'] == 1, 'CONFIRMADO'] = ['1']
df.loc[df['RESULTADO_LAB'] == 3, 'SOSPECHOSO'] = ['1']

df["SOSPECHOSO"] = pd.to_numeric(df["SOSPECHOSO"])
df["CONFIRMADO"] = pd.to_numeric(df["CONFIRMADO"])


df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'])

df_by_date = df.groupby("FECHA_SINTOMAS",as_index=False).count()

df_by_date['CONFIRMADOS_ESTIMADOS'] = df_by_date['CONFIRMADO'] + (df_by_date['SOSPECHOSO'] * last_positivity)

df_by_date['CONFIRMADO'] = df_by_date['CONFIRMADO'] + df_by_date['CONFIRMADOS_ESTIMADOS']

first_case_date = df_by_date['FECHA_SINTOMAS'].iloc[0]

last_case_date = df_by_date['FECHA_SINTOMAS'].iloc[-1]

delta = (last_case_date) - first_case_date       # as timedelta

active_df = pd.DataFrame()

for i in range(delta.days + 1):
    day = first_case_date + timedelta(days=i)

    df_by_date['FECHA'] = day
    
    active_date_minus_14_days = day - timedelta(days=11)
    mask = (df_by_date['FECHA_SINTOMAS'] >= active_date_minus_14_days) & (df_by_date['FECHA_SINTOMAS'] <= day)
    filtered_df=df_by_date.loc[mask]
    
    filtered_df = filtered_df.groupby(["FECHA"],as_index=False).sum()
    active_df = active_df.append(filtered_df)

fig = px.line(active_df, active_df.FECHA, active_df.CONFIRMADO,
              title='Activos Mexico por Estado')
fig.show()